In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.saving import get_custom_objects
from tensorflow import keras
from tensorflow.keras import layers

Es werden zunächst die Dimensionen für den Input und die Anzahl der Output Neuronen definiert.
Danach wird der MNIST Datensatz geladen, welcher bereits in Keras enthalten ist (muss trotzdem gedownloaded werden, falls noch nicht geschehen).

In [2]:
num_classes = 10
input_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

Die Daten werden zunächst auf Werte zwischen 0 und 1 normalisiert.
Danach wird die Form der Eingabedaten angepasst, sodass diese die im input_shape spezifizierte Form haben.
Die Labels werden ebenfalls angepasst. Hier wird ein One-Hot-Encoding durchgeführt. Danach haben unsere y-Daten 10 Spalten, mit den 2 möglichen Werten {0,1}.

In [3]:
x_train = x_train.astype("float32") /255
x_test = x_test.astype("float32") /255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Wir definieren ein Modell mit 2 Convolutional Layern, welche jeweils von einem Max-Pooling Layer gefolgt werden. Zum Schluss kommt noch ein Flatten Layer, welcher die entstandenen mehrdimensionalen Filter in einen eindimmensionalen Vektor umwandelt. Danach werden die Eindimensionalen Daten in den Fully-Connected Output Layer geführt.

In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

Wir definieren unsere Batchsize und die Anzahl der Epochs, welche wir durchführen wollen. Außerdem geben wir für unser Modell noch eine Kostenfunktion, den Optimierer, welcher verwendet werden soll und die Metriken, welche wir messen wollen an.

In [5]:
batch_size = 128
epochs = 15
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Wir starten das Training unseres Modells auf den Trainingsdaten mit der definierten Batchsize und führen das Training so oft aus wie wir dies in der Variable Epochs definiert haben.

In [6]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 16s 37ms/step - loss: 0.3718 - accuracy: 0.8865 - val_loss: 0.0839 - val_accuracy: 0.9770
Epoch 2/15
422/422 [==============================] - 15s 37ms/step - loss: 0.1158 - accuracy: 0.9640 - val_loss: 0.0581 - val_accuracy: 0.9847
Epoch 3/15
422/422 [==============================] - 16s 37ms/step - loss: 0.0872 - accuracy: 0.9733 - val_loss: 0.0488 - val_accuracy: 0.9877
Epoch 4/15
422/422 [==============================] - 16s 38ms/step - loss: 0.0739 - accuracy: 0.9769 - val_loss: 0.0458 - val_accuracy: 0.9870
Epoch 5/15
422/422 [==============================] - 16s 37ms/step - loss: 0.0665 - accuracy: 0.9793 - val_loss: 0.0408 - val_accuracy: 0.9893
Epoch 6/15
422/422 [==============================] - 16s 38ms/step - loss: 0.0588 - accuracy: 0.9816 - val_loss: 0.0381 - val_accuracy: 0.9897
Epoch 7/15
422/422 [==============================] - 16s 38ms/step - loss: 0.0534 - accuracy: 0.9826 - val_loss: 0.0352 - val_accuracy:

Wir können über die save() Funktion unsere Modelle ganz einfach in einem angegebenen Verzeichnis abspeichern.

In [7]:
model.save("./model")

INFO:tensorflow:Assets written to: ./model\assets


INFO:tensorflow:Assets written to: ./model\assets


Wir definieren eine Funktion, welche ein Polynom 2. Grades definiert. Danach fügen wir dieses in das get_custom_objects Dictionary von Keras hinzu.

In [8]:
def square_activation(x):
    return x*x + x
get_custom_objects().update({"square": layers.Activation(square_activation)})

In [9]:
get_custom_objects().update({"square": layers.Activation(square_activation)})

Wir definieren ein zweites Modell mit der selbstdefinierten quadratischen Funktion als Aktivierungsfunktion. Das Modell besitzt den gleichen Aufbau wie das erste Modell.

In [10]:
model2 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="square"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3,3), activation="square"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [11]:
model2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 20s 47ms/step - loss: 0.3013 - accuracy: 0.9068 - val_loss: 0.0682 - val_accuracy: 0.9818
Epoch 2/15
422/422 [==============================] - 20s 46ms/step - loss: 0.0949 - accuracy: 0.9711 - val_loss: 0.0496 - val_accuracy: 0.9862
Epoch 3/15
422/422 [==============================] - 20s 47ms/step - loss: 0.0682 - accuracy: 0.9789 - val_loss: 0.0401 - val_accuracy: 0.9885
Epoch 4/15
422/422 [==============================] - 20s 47ms/step - loss: 0.0568 - accuracy: 0.9820 - val_loss: 0.0372 - val_accuracy: 0.9900
Epoch 5/15
422/422 [==============================] - 20s 47ms/step - loss: 0.0493 - accuracy: 0.9844 - val_loss: 0.0344 - val_accuracy: 0.9892
Epoch 6/15
422/422 [==============================] - 20s 47ms/step - loss: 0.0429 - accuracy: 0.9861 - val_loss: 0.0308 - val_accuracy: 0.9913
Epoch 7/15
422/422 [==============================] - 20s 48ms/step - loss: 0.0406 - accuracy: 0.9868 - val_loss: 0.0285 - val_accuracy:

In [13]:
model2.save("./model_square")

INFO:tensorflow:Assets written to: ./model_square\assets


INFO:tensorflow:Assets written to: ./model_square\assets


Über die load_model() Funktion können wir abgespeicherte Modelle ganz einfach laden.

In [14]:
#model = keras.models.load_model("./model")
#model = keras.models.load_model("./model_square")

Evaluierung der beiden Modelle über die evaluate() Methode der Keras model Klasse

In [15]:
score = model.evaluate(x_test, y_test, verbose=0)
score2 = model2.evaluate(x_test, y_test, verbose=0)

In [16]:
print("Model 1:")
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("Model 2:")
print("Test loss:", score2[0])
print("Test accuracy:", score2[1])

Model 1:
Test loss: 0.02409188449382782
Test accuracy: 0.9922000169754028
Model 2:
Test loss: 0.023351045325398445
Test accuracy: 0.9919999837875366
